# 2.3 聚合平台统一调用

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/02-ai-landscape/aggregators.ipynb)

**预计 API 费用：$0**（使用 OpenRouter 的免费模型测试）

本 Notebook 演示如何使用 OpenRouter 聚合平台，用一套代码调用多个模型，并实现模型降级（fallback）和异步多模型对比。

## 安装依赖

In [ ]:
!pip install -q openai

## 配置 OpenRouter API Key

1. 前往 [OpenRouter](https://openrouter.ai/) 注册
2. 在 [Keys](https://openrouter.ai/keys) 页面创建 API Key
3. （可选）充值最低 $5，或使用免费模型测试

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENROUTER_API_KEY"):
    os.environ["OPENROUTER_API_KEY"] = getpass("请输入 OpenRouter API Key: ")

## 基础用法：用 OpenAI SDK 调用 OpenRouter

OpenRouter 完全兼容 OpenAI API 格式，只需修改 `base_url` 和 `model` 参数。

In [ ]:
from openai import OpenAI

# 创建 OpenRouter 客户端
client = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

# 调用 GPT-4o-mini
response = client.chat.completions.create(
    model="openai/gpt-4o-mini",  # 格式：提供商/模型名
    messages=[{"role": "user", "content": "用一句话介绍 Python"}],
)

print("【OpenAI GPT-4o-mini】")
print(response.choices[0].message.content)

## 一套代码调用多个模型

通过改变 `model` 参数，可以无缝切换不同提供商的模型。

In [ ]:
question = "用一句话解释什么是 API？"

models = [
    "openai/gpt-4o-mini",
    "google/gemini-2.0-flash-exp:free",  # 免费模型
    "meta-llama/llama-3.2-3b-instruct:free",  # 免费模型
]

print(f"问题：{question}\n")
print("=" * 80)

for model in models:
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": question}],
        )
        
        print(f"\n【{model}】")
        print(response.choices[0].message.content)
        print(f"Tokens: {response.usage.total_tokens}")
    
    except Exception as e:
        print(f"\n【{model}】")
        print(f"❌ 调用失败：{e}")

## 模型自动降级（Fallback Pattern）

在生产环境中，主模型可能因为限流、服务中断等原因调用失败。我们可以实现自动降级到备选模型。

In [ ]:
def chat_with_fallback(user_message, models, timeout=10):
    """
    优先使用高质量模型，失败时自动降级到备选模型
    
    Args:
        user_message: 用户消息
        models: 模型列表，按优先级排序
        timeout: 单次调用超时时间（秒）
    
    Returns:
        (answer, model_used) 或 (None, None)
    """
    for model in models:
        try:
            print(f"尝试调用：{model}")
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": user_message}],
                timeout=timeout,
            )
            
            answer = response.choices[0].message.content
            print(f"✅ 成功！使用模型：{model}\n")
            
            return answer, model
        
        except Exception as e:
            print(f"❌ 失败：{str(e)[:100]}\n")
            continue
    
    print("⚠️  所有模型均调用失败")
    return None, None

In [ ]:
# 测试自动降级
fallback_models = [
    "openai/gpt-4o",                        # 主力：质量最高但最贵
    "openai/gpt-4o-mini",                   # 备选 1：性价比高
    "google/gemini-2.0-flash-exp:free",     # 备选 2：免费
    "meta-llama/llama-3.2-3b-instruct:free",  # 备选 3：免费备份
]

answer, model_used = chat_with_fallback(
    "用一句话解释什么是递归",
    fallback_models
)

if answer:
    print("=" * 80)
    print(f"最终使用模型：{model_used}")
    print(f"回答：{answer}")

## 异步并发：同时对比多个模型

用 `asyncio` 并发调用多个模型，快速对比回答质量。

In [ ]:
import asyncio
from openai import AsyncOpenAI

async def compare_models_async(question, models):
    """
    并发调用多个模型，对比回答
    """
    client = AsyncOpenAI(
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url="https://openrouter.ai/api/v1",
    )
    
    async def call_model(model):
        try:
            response = await client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": question}],
                timeout=30,
            )
            
            return {
                "model": model,
                "answer": response.choices[0].message.content,
                "tokens": response.usage.total_tokens,
                "success": True,
            }
        
        except Exception as e:
            return {
                "model": model,
                "error": str(e)[:100],
                "success": False,
            }
    
    # 并发调用所有模型
    tasks = [call_model(model) for model in models]
    results = await asyncio.gather(*tasks)
    
    return results

In [ ]:
# 测试：同时对比 5 个模型
test_question = "用一个生活中的类比，解释什么是 Docker？"

comparison_models = [
    "openai/gpt-4o-mini",
    "google/gemini-2.0-flash-exp:free",
    "meta-llama/llama-3.2-3b-instruct:free",
    "mistralai/mistral-7b-instruct:free",
    "qwen/qwen-2.5-7b-instruct:free",
]

print(f"问题：{test_question}\n")
print("=" * 80)
print("正在并发调用 5 个模型...\n")

results = await compare_models_async(test_question, comparison_models)

# 显示结果
for result in results:
    print(f"\n【{result['model']}】")
    
    if result['success']:
        print(result['answer'])
        print(f"\nTokens: {result['tokens']}")
    else:
        print(f"❌ 调用失败：{result['error']}")

print("\n" + "=" * 80)
print("对比完成！")

## 智能路由：自动选择最优模型

OpenRouter 支持自动路由策略，可以根据价格、速度、质量自动选择模型。

In [ ]:
# 自动选择最便宜的模型
response = client.chat.completions.create(
    model="openrouter/auto",
    messages=[{"role": "user", "content": "你好"}],
    extra_body={
        "route": "cheapest",  # 按价格选（cheapest / fastest / best）
    },
)

print("【自动路由：最便宜】")
print(f"使用的模型：{response.model}")
print(f"回答：{response.choices[0].message.content}")

## 动手练习

1. 尝试添加更多模型到对比列表中（查看 [OpenRouter 模型列表](https://openrouter.ai/models)）
2. 实现一个智能选择器：根据问题类型（编程、写作、数学）自动选择最合适的模型
3. 测试降级策略的响应时间：主模型失败后多久能切换到备选模型？

## 小结

- **聚合平台解决了什么**：用一套代码（OpenAI SDK）调用 200+ 模型，降低维护成本
- **Fallback Pattern**：实现高可用性，主模型失败时自动降级
- **异步并发**：快速对比多个模型的回答质量
- **智能路由**：根据价格、速度、质量自动选择最优模型

**关键要点：**
- 只需改 `base_url` 和 `model` 参数，无需修改其他代码
- 模型名格式：`提供商/模型名`（如 `openai/gpt-4o-mini`）
- OpenRouter 支持 200+ 模型，价格与官方相同

下一步：学习如何在本地部署开源模型，实现零成本运行 → [2.4 本地模型部署](./local_deployment.ipynb)